In [250]:
import datetime
import os
import requests
import sys
import time
import random

In [251]:
URL = "https://pprdv.interieur.gouv.fr/booking/create/989"
URL_1 = "https://pprdv.interieur.gouv.fr/booking/create/989/1"

planning_num = [990 , 1000, 1001, 1002, 1003, 1005, 1028, 1029]
index = list(range(0,8))
text_on_page= ['1,2 et 3' , '4', "6", '5', '7', '8', '9', '10']

# We need to look like a normal browser because otherwise we are blocked
headers = {
    'User-Agent': 'Mozilla/5.0 (X11; Fedora; Linux x86_64; rv:64.0) ' \
    'Gecko/20100101 Firefox/64.0' \
    'Chrome/69.0.3497.100' \
    'Safari/537.36' 
}
checkPhrase = "Il n'existe plus de plage horaire libre pour votre " 
checkError = "The server returned an invalid or incomplete response."
   
# "demande de rendez-vous." \

thisDir = os.path.dirname('./')

In [252]:
headerlist = ["Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/62.0.3202.94 Safari/537.36",
     "Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36 OPR/43.0.2442.991",
           "Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.87 Safari/537.36 OPR/42.0.2393.94",
           "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.78 Safari/537.36 OPR/47.0.2631.39",
           "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36",
           "Mozilla/5.0 (Windows NT 5.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36",
           "Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.87 Safari/537.36",
           "Mozilla/5.0 (Windows NT 6.1; WOW64; rv:54.0) Gecko/20100101 Firefox/54.0",
           "Mozilla/5.0 (Windows NT 10.0; WOW64; rv:54.0) Gecko/20100101 Firefox/54.0",
           "Mozilla/5.0 (Windows NT 6.1; WOW64; rv:56.0) Gecko/20100101 Firefox/56.0",
           "Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; rv:11.0) like Gecko"]

In [253]:
def checkPlacesAreAlreadyTaken():
    with requests.Session() as s:
        #user_agent = random.choice(headerlist)
        #headers = {'User-Agent': user_agent}
        
        r = s.get(URL, headers=headers)
        r = s.post(URL, headers=headers,
                   data={'condition':'on',
                         'nextButton':'Effectuer+une+demande+de+rendez-vous'})
        r = s.get(URL_1, headers=headers)
        ram = random.choice(index)
        r = s.post(URL_1, headers=headers,
                      data={'planning': planning_num[ram] ,
                            'nextButton':'Etape+suivante'})
        
        print("Remise de titre guichet "+ text_on_page[ram])
       # print(headers) 
       # print(r.text)
    if checkError in r.text :
        return "Error"
    else:
        return checkPhrase in r.text

In [254]:
def writeResults(placesAreAlreadyTaken):
    """Log the result to STDOUT and to two different files."""
    now = datetime.datetime.now()
    print('{} All places are already taken: {}'.format(now, placesAreAlreadyTaken))
    if placesAreAlreadyTaken:
        with open(os.path.join(thisDir, 'placesAreAlreadyTaken.log'), 'a') as f:
            f.write('{} Places are taken\n'.format(now))
    elif placesAreAlreadyTaken == "Error" :
        with open(os.path.join(thisDir, 'placesAreAlreadyTaken.log'), 'a') as f:
            f.write('{} Page Error\n'.format(now))
    else:
        with open(os.path.join(thisDir, 'placeIsAvailable.log'), 'a') as f:
            f.write('{} At least one place is available!\n'.format(now))

In [255]:
def playSound():
    """Emit three terminal beeps within 1.5 s."""
    for i in range(3):
        sys.stdout.write('\a')
        sys.stdout.flush()
        time.sleep(0.5)

In [256]:
def main():
    print('Starting to monitor the reservation system.')
    print('Press CTRL+C to abort...\n')
    while True:
        try:
            placesAreAlreadyTaken = checkPlacesAreAlreadyTaken()
        except requests.exceptions.RequestException as e:
            sys.stderr.write('** Error while querying site: {}\n'.format(e))
        else:
            if not placesAreAlreadyTaken:
                playSound()
            writeResults(placesAreAlreadyTaken)
        time.sleep(180)  

In [ ]:
if __name__ == '__main__':
    sys.exit(main())

Starting to monitor the reservation system.
Press CTRL+C to abort...



** Error while querying site: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
